In [144]:
import os
import numpy as np
import tensorflow as tf
from numpy import genfromtxt
from tensorflow.python.util import compat

In [145]:
data = genfromtxt('/root/bandit/bandit_data.csv',
                  delimiter=',', skip_header = 1, dtype = np.float32)

In [146]:
labels = data[:,5]
num_labels = len(set(labels))
labels = (np.arange(num_labels) == np.array(labels)[:,None]).astype(np.float32)

In [147]:
features = data[:,1:3]
reward = data[:,3:4]
probability = data[:,4:5]
feature_size = 2

In [143]:
reward.shape

(10000, 1)

In [168]:
x = tf.placeholder(tf.float32, [None, feature_size])

W = tf.Variable(tf.truncated_normal([feature_size, num_labels]))
b = tf.Variable(tf.truncated_normal([num_labels]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

In [175]:
y_ = tf.placeholder(tf.float32, [None, 3])
reward_ = tf.placeholder(tf.float32, [None, 1])
probability_ = tf.placeholder(tf.float32, [None, 1])

arm_choice = tf.argmax(y, 1)
input_label = tf.argmax(y_, 1)

arm_equals_input = tf.cast(tf.equal(arm_choice, input_label), tf.float32)
inverse_prop_score = -tf.divide(reward_, probability_)

ips_output = tf.multiply(arm_equals_input, inverse_prop_score)
cost = tf.reduce_mean(ips_output)

optimizer = tf.contrib.opt.ScipyOptimizerInterface(cost, 
                                                   options={'maxiter': 100000},
                                                  var_list = [W, b])

In [176]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    optimizer.minimize(sess, feed_dict = {x: features, y_: labels, reward_: reward, probability_: probability})
    
    curr_W, curr_b, curr_loss  = sess.run([W, b, cost], {x: features, y_: labels, reward_: reward, probability_: probability})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

# I don't think this is doing anything...

INFO:tensorflow:Optimization terminated with:
  Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  Objective function value: -0.279767
  Number of iterations: 0
  Number of functions evaluations: 1
W: [[ 0.56017971 -0.514314   -0.24636835]
 [-1.19200516  0.15526015 -0.03196505]] b: [-0.06614329  0.62429565 -0.76556677] loss: -0.279767
